In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from typing import *
from pprint import pprint

import bigbench.api.results as bb

from lass.log_handling import LogLoader, TaskLog
from lass.datasets import split_task_level, analyse, merge, huggingfaceify

In [2]:
loader = (LogLoader(logdir = Path('../artifacts/logs'))
        .with_tasks('paper-full')
        .with_model_families(['BIG-G T=0'])
        .with_model_sizes(['128b'])
        .with_shots([0,1,2,3])
        .with_query_types([bb.MultipleChoiceQuery])
)

train, test = split_task_level(loader, seed=42, test_fraction=0.2)

In [3]:
stats = merge(analyse(train), analyse(test), 'train', 'test')
del stats['task_names'] # Can't read anything anymore otherwise
pprint(stats)
train.head(1)

{'metrics': {'conf-absolute': {'roc_auc': {'test': 0.6220219922369651,
                                           'train': 0.578043799158497}},
             'conf-normalized': {'roc_auc': {'test': 0.6217446124045184,
                                             'train': 0.6021936348361989}},
             'task-acc': {'test': 0.3242025640312578,
                          'train': 0.36355745077933843}},
 'stats': {'n_instances': {'test': 35959, 'train': 184323},
           'n_instances_nonbinary': {'test': 461, 'train': 981},
           'n_tasks': {'test': 23, 'train': 95}}}


,input,targets,scores,target_values,correct,absolute_scores,normalized_scores,metrics,task,shots
0,\nIn the SIT-adversarial world a structure is ...,[There is at least one triangle pointing down....,"[-9.594904899597168, -7.830936431884766, -8.70...","{'There are at least two blue pieces. ': 0, 'T...",1.0,"[-9.594904899597168, -7.830936431884766, -8.70...","[-2.8301148414611816, -1.0661463737487793, -1....",{'calibration_multiple_choice_brier_score': 0....,symbol_interpretation,0


In [4]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

import torch

print(torch.cuda.device_count())

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


1


In [5]:
dataset = huggingfaceify(train, test)
dataset['train'][0]


{'text': '\nIn the SIT-adversarial world a structure is a sequence of six emojis.\nHereafter are reported the emojis used along with their descriptions.\n 🔺 is a red circle;\n 🟦 is a blue circle;\n 🔴 is a yellow circle;\n 🟥 is a red triangle pointing up;\n 🟨 is a red triangle pointing down;\n 🔻 is a red square;\n 🟡 is a blue square;\n _ is a yellow square;\n 🔵 is an empty space.\n\nChoose the sentence consistent with the structure 🟥 🔻 🔺 🟡 🟥 🟨 and not consistent with 🔺 🟡 🟡 🟨 🟦 _:\n\n  choice: There are at least two triangles.\n\n  choice: There is at least one triangle.\n\n  choice: There are at least two yellow squares.\n\n  choice: There are at least two blue pieces.\n\n  choice: There is at least one triangle pointing down.\n\nA: ',
 'label': 1}

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")
    return tokenizer(examples["text"], padding="max_length", truncation=True)
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=1024) # longformer
    # return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="np") #gpt-2
    # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=2048) # xlnet

# tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Parameter 'function'=<function tokenize_function at 0x7f0964dd91b0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/185 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

In [7]:
train_dataset = tokenized_datasets["train"].shuffle(seed=42) #.select(range(50))
eval_dataset = tokenized_datasets["test"].shuffle(seed=42) #.select(range(50))
len(train_dataset), len(eval_dataset)

(184323, 35959)

In [8]:
import wandb
%env WANDB_PROJECT=lass
%env WANDB_LOG_MODEL=true
%env TOKENIZERS_PARALLELISM=true
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


env: WANDB_PROJECT=lass
env: WANDB_LOG_MODEL=true
env: TOKENIZERS_PARALLELISM=true


wandb: Currently logged in as: wschella. Use `wandb login --relogin` to force relogin


True

In [9]:
from transformers.models.auto.modeling_auto import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import BertModel, BertConfig
import numpy as np
from datasets import load_metric
import scipy
import torch
import lass, lass.metrics.hf

# model = BertModel(BertConfig.from_pretrained("bert-base-cased"))
model = AutoModelForSequenceClassification.from_pretrained("albert-base-v2")
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=2)
# model.config.pad_token_id = model.config.eos_token_id
# model = AutoModelForSequenceClassification.from_pretrained("./test_trainer/checkpoint-13500", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("../artifacts/assessors/bert-bs32/checkpoint-3000", num_labels=2)

training_args = TrainingArguments(
    output_dir="albert-bs32-allsh-wd-task-split",
    run_name="albert-bs32-allsh-wd-task-split",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    report_to="wandb",
    num_train_epochs=10,
    # max_steps=17277 - 3000,
)

compute_metrics = lass.metrics.hf.get_metric_computer([
  "accuracy",
  "precision",
  "recall",
  "f1",
  "roc_auc",
  "brier_score",
])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
torch.cuda.empty_cache()
trainer.train()

wandb.finish()

The following columns in the training set don't have a corresponding argument in `AlbertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `AlbertForSequenceClassification.forward`,  you can safely ignore this message.


/home/wout/pp/lass/.env/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****


  Num examples = 184323


  Num Epochs = 10


  Instantaneous batch size per device = 32


...



Training completed. Do not forget to share your model on huggingface.co/models =)




Saving model checkpoint to /tmp/tmplrzjzii9


Configuration saved in /tmp/tmplrzjzii9/config.json


Model weights saved in /tmp/tmplrzjzii9/pytorch_model.bin


eval/accuracy,█▁▅▆▃▄▂▂▂▁
eval/brier_score,▁▁▁▂▃▄▅▆▇█
eval/f1,▁█▆▅█▇██▇▇
eval/loss,▁▁▁▂▃▃▄▆▇█
eval/precision,█▁▃▃▂▃▂▂▁▁
eval/recall,▁▇▅▄▇▇██▆▇
eval/roc_auc,█▇▆▅▅▄▂▂▂▁
eval/runtime,▁▁▁█▁▁█▁▁▁
eval/samples_per_second,███▁▇█▁███
eval/steps_per_second,███▁▇█▁███
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███


In [11]:
import wandb
# wandb.finish()